In [0]:
!pip3 -qq install torch==0.4.1
!pip -qq install torchtext==0.3.1
!pip -qq install gensim==3.6.0
!pip -qq install pyldavis==2.1.2
!pip -qq install attrs==18.2.0
!wget -qq --no-check-certificate 'https://drive.google.com/uc?export=download&id=1OIU9ICMebvZXJ0Grc2SLlMep3x9EkZtz' -O perashki.txt
!wget -qq --no-check-certificate 'https://drive.google.com/uc?export=download&id=1v66uAEKL3KunyylYitNKggdl2gCeYgZZ' -O poroshki.txt
!git clone https://github.com/UniversalDependencies/UD_Russian-SynTagRus.git
!wget -qq https://raw.githubusercontent.com/DanAnastasyev/neuromorphy/master/neuromorphy/train/corpus_iterator.py

tcmalloc: large alloc 1073750016 bytes == 0x58bde000 @  0x7f8f35c4a2a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
Cloning into 'UD_Russian-SynTagRus'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 284 (delta 2), reused 5 (delta 1), pack-reused 276
Receiving objects: 100% (284/284), 154.19 MiB | 23.38 MiB/s, done.
Resolving deltas: 100% (175/175), done.


In [0]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


if torch.cuda.is_available():
    from torch.cuda import FloatTensor, LongTensor
    DEVICE = torch.device('cuda')
else:
    from torch import FloatTensor, LongTensor
    DEVICE = torch.device('cpu')

np.random.seed(42)

# Word-Level Text Generation

Сегодня занимаемся, в основном, тем, что генерируем *пирожки* и *порошки*.

*(Данные без спросу скачаны с сайта http://poetory.ru)*

Пирожки - это вот:

In [0]:
!head perashki.txt

старик вытягивает сети
они пусты и лишь в конце
записка рыба недоступна
или вне действия сети

олег адепт шизофрении
шагает бодро из окна
и жызнь летит перед глазами
да не одна а сразу две



Порошки вот:

In [0]:
!head poroshki.txt

кто любит цоя кто покушать
кто на рассвете пенье птиц
а я люблю в коротких платьях
физлиц

твой монолог так гениален
что я чуть не открыла дверь
но станиславский тихо сверху
не верь



Не перепутайте!

Вообще, пирожок - это четверостишие, написанное четырехстопным ямбом по схеме 9-8-9-8. У порошка схема 9-8-9-2.

In [0]:
vowels = 'ёуеыаоэяию'

odd_pattern = '-+-+-+-+-'
even_pattern = '-+-+-+-+'

In [0]:
def count_vowels(word):
    return len([letter for letter in word if letter in vowels])

def get_pattern(token):
    pattern = []
    counter = 0
    for word in token:
        syl = count_vowels(word)
        pattern.append(odd_pattern[counter : counter + syl] if counter + syl < len(odd_pattern) else '<pad>')
        counter += syl
    return pattern

get_pattern(['покушать'])

['-+-']

Считываем данные:

In [0]:
def read_poem(path):
    poem = []
    poem_pattern = []
    with open(path, encoding='utf8') as f:
        for line in f:
            line = line.rstrip()
            pattern = get_pattern(line.split())
            if len(line) == 0:
                yield poem, poem_pattern
                poem = []
                poem_pattern = []
                continue
            
            poem.extend(line.split() + ['\\n'])
            poem_pattern.extend(pattern + ['\\n'])
            
perashki = list(read_poem('perashki.txt'))
poroshki = list(read_poem('poroshki.txt'))

In [0]:
from torchtext.data import Field, Example, Dataset, BucketIterator

text_field = Field(init_token='<s>', eos_token='</s>')
pattern_field = Field(init_token='<s>', eos_token='</s>')
        
fields = [('text', text_field), ('pattern', pattern_field)]
examples = [Example.fromlist([poem, pattern], fields) for poem, pattern in perashki]
dataset = Dataset(examples, fields)

text_field.build_vocab(dataset, min_freq=7)
pattern_field.build_vocab(dataset, min_freq=7)

print('Vocab size =', len(text_field.vocab))
print('Pattern vocab size =', len(pattern_field.vocab))
train_dataset, test_dataset = dataset.split(split_ratio=0.9)

train_iter, test_iter = BucketIterator.splits(datasets=(train_dataset, test_dataset), batch_sizes=(64, 128), 
                                              shuffle=True, device=DEVICE, sort=False)

Vocab size = 13070
Pattern vocab size = 21


In [0]:
pattern_field.vocab.itos

['<unk>',
 '<pad>',
 '<s>',
 '</s>',
 '\\n',
 '-',
 '-+',
 '+',
 '+-',
 '-+-',
 '+-+',
 '',
 '-+-+',
 '+-+-',
 '+-+-+',
 '-+-+-',
 '-+-+-+',
 '+-+-+-',
 '-+-+-+-',
 '-+-+-+-+',
 '+-+-+-+']

Построим датасет для порошков:

In [0]:
from torchtext.data import Field, Example, Dataset, BucketIterator

text_field = Field(init_token='<s>', eos_token='</s>')
pattern_field = Field(init_token='<s>', eos_token='</s>')
        
fields = [('text', text_field), ('pattern', pattern_field)]
examples = [Example.fromlist([poem, pattern], fields) for poem, pattern in poroshki]
dataset = Dataset(examples, fields)

text_field.build_vocab(dataset, min_freq=7)
pattern_field.build_vocab(dataset, min_freq=7)

print('Vocab size =', len(text_field.vocab))
print('Pattern vocab size =', len(pattern_field.vocab))

train_dataset, test_dataset = dataset.split(split_ratio=0.9)

train_iter, test_iter = BucketIterator.splits(datasets=(train_dataset, test_dataset), batch_sizes=(32, 128), 
                                              shuffle=True, device=DEVICE, sort=False)

Vocab size = 6298
Pattern vocab size = 15


In [0]:
batch = next(iter(train_iter))

**Задание** Напишите класс языковой модели.

In [0]:
class LMModel(nn.Module):
    def __init__(self, vocab_size, pattern_vocab_size=None, emb_dim=256, pattern_emb_dim=64, lstm_hidden_dim=256, num_layers=1):
        super().__init__()

        self._emb = nn.Embedding(vocab_size, emb_dim)
        if pattern_vocab_size is not None:
            self._pattern_emb = nn.Embedding(pattern_vocab_size, pattern_emb_dim)
            self._rnn = nn.LSTM(input_size=emb_dim + pattern_emb_dim, hidden_size=lstm_hidden_dim)
            self._pattern_out_layer = nn.Linear(lstm_hidden_dim, pattern_vocab_size)
        else:
            self._rnn = nn.LSTM(input_size=emb_dim, hidden_size=lstm_hidden_dim)
        
        self._out_layer = nn.Linear(lstm_hidden_dim, vocab_size)
        
        self._init_weights()

    def _init_weights(self, init_range=0.1):
        self._emb.weight.data.uniform_(-init_range, init_range)
        self._out_layer.bias.data.zero_()
#         if pattern_vocab_size is not None:
#             self._out_layer.weight.data.uniform_(-init_range, init_range)
#         else:
        self._out_layer.weight = self._emb.weight
        

    def forward(self, inputs, pattern_inputs=None, hidden=None):
        embeds = self._emb(inputs)
        
        if pattern_inputs is not None:
            pattern_embeds = self._pattern_emb(pattern_inputs)
            embeds = torch.cat((embeds, pattern_embeds), 2)
            
        outputs, hidden = self._rnn(embeds, hidden)
        
        text_outputs = self._out_layer(outputs)
        if pattern_inputs is not None:
            pattern_outputs = self._pattern_out_layer(outputs)
            
            return text_outputs, pattern_outputs, hidden
        
        return text_outputs, hidden

In [0]:
model = LMModel(vocab_size=len(train_iter.dataset.fields['text'].vocab), pattern_vocab_size=len(train_iter.dataset.fields['pattern'].vocab)).to(DEVICE)

model(batch.text, batch.pattern)[0].shape

torch.Size([29, 64, 13070])

**Задание** Добавьте подсчет потерей с маскингом паддингов.

In [0]:
import math
from tqdm import tqdm
tqdm.get_lock().locks = []


def do_epoch(model, criterion, data_iter, unk_idx, pad_idx, use_patterns=False, optimizer=None, name=None):
    epoch_loss = 0
    
    is_train = not optimizer is None
    name = name or ''
    model.train(is_train)
    
    batches_count = len(data_iter)
    
    with torch.autograd.set_grad_enabled(is_train):
#         with tqdm(total=batches_count) as progress_bar:
            for i, batch in enumerate(data_iter):    
                labels = batch.text[1:, :]
                labels = labels.view(-1)
                
                if use_patterns:
                    pattern_labels = batch.pattern[1:, :]
                    pattern_labels = pattern_labels.view(-1)
                
                    logits, pattern_logits, _ = model(batch.text, batch.pattern)
                    
                    pattern_logits = pattern_logits[:-1, :, :]
                    pattern_logits = pattern_logits.view(-1, pattern_logits.shape[-1])
                    
                    mask = ((pattern_labels != 0) * (pattern_labels != 1)).float()  # fixme
                    
                    pattern_loss = torch.sum(criterion(pattern_logits, pattern_labels.view(-1)) * mask) / torch.sum(mask)
                else:
                    logits, _ = model(batch.text)
                logits = logits[:-1, :, :]
                logits = logits.view(-1, logits.shape[-1])
                
                mask = ((labels != pad_idx) * (labels != unk_idx)).float()
                
                loss = torch.sum(criterion(logits, labels.view(-1)) * mask) / torch.sum(mask)
                if use_patterns:
                    loss = (loss + pattern_loss) / 2
                
                epoch_loss += loss.item()

                if optimizer:
                    optimizer.zero_grad()
                    loss.backward()
                    nn.utils.clip_grad_norm_(model.parameters(), 1.)
                    optimizer.step()

#                 progress_bar.update()
#                 progress_bar.set_description('{:>5s} Loss = {:.5f}, PPX = {:.2f}'.format(name, loss.item(), 
#                                                                                          math.exp(loss.item())))
                
#             progress_bar.set_description('{:>5s} Loss = {:.5f}, PPX = {:.2f}'.format(
#                 name, epoch_loss / batches_count, math.exp(epoch_loss / batches_count))
#             )
#             progress_bar.refresh()
            print('{:>5s} Loss = {:.5f}, PPX = {:.2f}'.format(name, epoch_loss / batches_count, math.exp(epoch_loss / batches_count)))
                    
    return epoch_loss / batches_count


def fit(model, criterion, optimizer, train_iter, use_patterns=False, epochs_count=1, unk_idx=0, pad_idx=1, val_iter=None):
    best_val_loss = None
    for epoch in range(epochs_count):
        name_prefix = '[{} / {}] '.format(epoch + 1, epochs_count)
        train_loss = do_epoch(model, criterion, train_iter, unk_idx, pad_idx, use_patterns, optimizer, name_prefix + 'Train:')
        
        if not val_iter is None:
            val_loss = do_epoch(model, criterion, val_iter, unk_idx, pad_idx, use_patterns, None, name_prefix + '  Val:')
            
            if best_val_loss and val_loss > best_val_loss:
                optimizer.param_groups[0]['lr'] /= 4.
                print('Optimizer lr = {:g}'.format(optimizer.param_groups[0]['lr']))
            else:
                best_val_loss = val_loss
        print()
        generate(model, use_patterns)
        print()

**Задание** Напишите функцию-генератор для модели.

In [0]:
def sample(probs, temp):
    probs = F.log_softmax(probs.squeeze(), dim=0)
    probs = (probs / temp).exp()
    probs /= probs.sum()
    probs = probs.cpu().numpy()

    return np.random.choice(np.arange(len(probs)), p=probs)


def generate(model, use_patterns=False, temp=0.6):
    model.eval()
    with torch.no_grad():        
        prev_token = train_iter.dataset.fields['text'].vocab.stoi['<s>']
        end_token = train_iter.dataset.fields['text'].vocab.stoi['</s>']
        if use_patterns:
            prev_pattern_token = train_iter.dataset.fields['pattern'].vocab.stoi['<s>']
        
        new_line_token = train_iter.dataset.fields['text'].vocab.stoi['\\n']
        
        hidden = None
        for _ in range(150):
            if use_patterns:
                probs, pattern_probs, hidden = model(LongTensor([[prev_token]]), LongTensor([[prev_pattern_token]]), hidden)
            else:
                probs, hidden = model(LongTensor([[prev_token]]), None, hidden)
            prev_token = sample(probs[-1], temp)
            
            if use_patterns:
#                 word = train_dataset.fields['text'].vocab.itos[prev_token]
#                 pattern = get_pattern(word)
#                 prev_pattern_token = train_iter.dataset.fields['pattern'].vocab.stoi[pattern]
                
                prev_pattern_token = sample(pattern_probs[-1], temp)
                pattern = train_iter.dataset.fields['pattern'].vocab.itos[prev_pattern_token]
            
            if prev_token == end_token:
                return
            if prev_token != new_line_token and use_patterns:
                print(train_dataset.fields['text'].vocab.itos[prev_token] + " (" + train_iter.dataset.fields['pattern'].vocab.itos[prev_pattern_token] + ")", end=' ')
            elif prev_token != new_line_token:
                print(train_dataset.fields['text'].vocab.itos[prev_token], end=' ')
            else:
                print()
            
                
generate(model)

я вас люблю сказал аркадий 
и в эти несколько часов 
и вдруг один из них с тобою 
мне в сердце нечего любовь 


In [0]:
model = LMModel(vocab_size=len(train_iter.dataset.fields['text'].vocab)).to(DEVICE) # pattern_vocab_size=len(train_iter.dataset.fields['pattern'].vocab)

pad_idx = train_iter.dataset.fields['text'].vocab.stoi['<pad>']
unk_idx = train_iter.dataset.fields['text'].vocab.stoi['<unk>']
criterion = nn.CrossEntropyLoss(reduction='none').to(DEVICE)

optimizer = optim.SGD(model.parameters(), lr=20., weight_decay=1e-6)

fit(model, criterion, optimizer, train_iter, use_patterns=False, epochs_count=300, unk_idx=unk_idx, pad_idx=pad_idx, val_iter=test_iter)

[1 / 300] Train: Loss = 5.60144, PPX = 270.82
[1 / 300]   Val: Loss = 5.04384, PPX = 155.06

олег на вспомнив в руках и в поле 
сказал я в квартиру и в груди 
и по то что я в гробу на поле 
не я не знаю как не на этом 

[2 / 300] Train: Loss = 4.88843, PPX = 132.75
[2 / 300]   Val: Loss = 4.76736, PPX = 117.61

конец когда ты спросила мне 
что я не видел как я так 
а ты уже тебя и я 
я всё же я люблю люблю 

[3 / 300] Train: Loss = 4.59648, PPX = 99.13
[3 / 300]   Val: Loss = 4.57564, PPX = 97.09

нет ничего не видел смысла 
я не могу я помню это 
что ты не будет никогда 
и не выходит за тобой 

[4 / 300] Train: Loss = 4.41046, PPX = 82.31
[4 / 300]   Val: Loss = 4.48467, PPX = 88.65

я не люблю тебя я умер 
и что то я не человек 
а ты мне нравится что время 
не будет так не я не ждал 

[5 / 300] Train: Loss = 4.26175, PPX = 70.93
[5 / 300]   Val: Loss = 4.43870, PPX = 84.67

в моей квартире все шаинский 
и смотрит в рот и говорит 
что он в метро пришли по плану 
и без цветов не отвеча

KeyboardInterrupt: ignored

In [0]:
model_with_patterns = LMModel(vocab_size=len(train_iter.dataset.fields['text'].vocab), pattern_vocab_size=len(train_iter.dataset.fields['pattern'].vocab)).to(DEVICE) # 

pad_idx = train_iter.dataset.fields['text'].vocab.stoi['<pad>']
unk_idx = train_iter.dataset.fields['text'].vocab.stoi['<unk>']
criterion = nn.CrossEntropyLoss(reduction='none').to(DEVICE)

optimizer = optim.SGD(model_with_patterns.parameters(), lr=20., weight_decay=1e-6)

fit(model_with_patterns, criterion, optimizer, train_iter, use_patterns=True, epochs_count=300, unk_idx=unk_idx, pad_idx=pad_idx, val_iter=test_iter)

[1 / 300] Train: Loss = 6.81910, PPX = 915.16
[1 / 300]   Val: Loss = 5.89147, PPX = 361.94

олег я с вами 
на твоих 
очках мне с ними 
я тут день с ним не не не же что есть не то же же всё что же с ним же же и же же ни не я как что же же же же день как пор не же меня 
я со думал 
из не с бутылкой 
с собой 
я мне на на вот же с тобою 
я чем нибудь 

[2 / 300] Train: Loss = 6.85604, PPX = 949.60
[2 / 300]   Val: Loss = 6.85798, PPX = 951.44
Optimizer lr = 5

когда у нас с тобою 
тот на столе 
я с горочки 

[3 / 300] Train: Loss = 5.26675, PPX = 193.79
[3 / 300]   Val: Loss = 5.21210, PPX = 183.48

вот вот я жена за них я наши не было 
а я в том же там не будет в дневнике 

[4 / 300] Train: Loss = 5.13893, PPX = 170.53
[4 / 300]   Val: Loss = 5.10208, PPX = 164.36

сегодня я не любишь 
а я не надо вышла 
так тот мне в неделю 
и молвил нет не надо 
а если больше на работу 

[5 / 300] Train: Loss = 5.02604, PPX = 152.33
[5 / 300]   Val: Loss = 4.99745, PPX = 148.04

в костюме все и в нём т

KeyboardInterrupt: ignored

In [0]:
model_with_patterns_and_common_loss = LMModel(vocab_size=len(train_iter.dataset.fields['text'].vocab), pattern_vocab_size=len(train_iter.dataset.fields['pattern'].vocab)).to(DEVICE) # 

pad_idx = train_iter.dataset.fields['text'].vocab.stoi['<pad>']
unk_idx = train_iter.dataset.fields['text'].vocab.stoi['<unk>']
criterion = nn.CrossEntropyLoss(reduction='none').to(DEVICE)

optimizer = optim.SGD(model_with_patterns_and_common_loss.parameters(), lr=20., weight_decay=1e-6)

fit(model_with_patterns_and_common_loss, criterion, optimizer, train_iter, use_patterns=True, epochs_count=300, unk_idx=unk_idx, pad_idx=pad_idx, val_iter=test_iter)

[1 / 300] Train: Loss = 3.34254, PPX = 28.29
[1 / 300]   Val: Loss = 3.02771, PPX = 20.65

олег (-+) в (-+) не (-+) к (-) идёш (+) в (-) женщин (+) а (\n) и (-) в (+) то (-+) не (-) знаю (+) и (-) то (+) что (-) просто () это (+) 
и (-+) на (-+) поедем (-+-) в (+) руке (\n) 
[2 / 300] Train: Loss = 2.94664, PPX = 19.04
[2 / 300]   Val: Loss = 2.89873, PPX = 18.15

как (-+-) мы (+-) как (+-) секса (+) такое () олегу (-+) же (-) умер () нам () рядом () нам (+) же () же (-+) же (\n) он () другим (-+) за (-+) прохожих (-+) я (-) с () кем (+) же () чувства (\n) 
[3 / 300] Train: Loss = 2.83143, PPX = 16.97
[3 / 300]   Val: Loss = 2.79544, PPX = 16.37

у (-+) меня (-+) уже (-+) не (-) любит (+-) секса () жить () людям () нам () так () нам () терапевт () вам () членом () вам () так () ты () вам () вам () вам () вам () так () мы () кем (-) пор (+) как (-) ждал (+) 
а () то (+-) я (+-) буду () так (+) же (\n) он (-) думал (+-+) что (-) я (+) мне (-) больше () вами (+) 
но (-) на (+) столе (-+-)

KeyboardInterrupt: ignored

**Задание** Добавьте маскинг `<unk>` токенов при тренировке модели.

## Улучшаем модель

### Tying input and output embeddings

В модели есть два эмбеддинга - входной и выходной. Красивая и полезная в жизни идея - учить только одну матрицу, расшаренную между ними: [Using the Output Embedding to Improve Language Models](http://www.aclweb.org/anthology/E17-2025).

От идеи одни плюсы: получается намного меньше обучаемых параметров и при этом достаточно заметно более высокое качество.

**Задание** Реализуйте это. Достаточно написать что-то типа этого в конструкторе:

`self._out_layer.weight = self._emb.weight`

### Добавление информации в выборку

Сейчас у нас каждое слово предствляется одним индексом. Модели очень сложно узнать, сколько в нем слогов - а значит, сложно генерировать корректное стихотворение.

На самом деле к каждому слову можно приписать кусочек из метрического шаблона:

![](https://hsto.org/web/59a/b39/bd0/59ab39bd020c49a78a12cbab62c80181.png =x200)

**Задание** Обновите функцию `read_poem`, пусть она генерирует два списка - список слов и список кусков шаблона.  
Добавьте в модель вход - последовательности шаблонов, конкатенируйте их эмбеддинги со словами.  
Дополнительная идея - заставьте модель угадывать, какой шаблон должен идти следующим (где-то половина будет подходящими, остальные - нет). Добавьте дополнительные потери от угадывания шаблона.

### Увеличиваем выборку

У нас есть выборка для пирожков, которая заметно больше.

**Задание** Обучитесь на ней.

### Transfer learning

Простой и приятный способ улучшения модели - сделать перенос обученной на большом корпусе модели на меньшего объема датасет.

Популярен этот способ больше в компьютерном зрении: [Transfer learning, cs231n](http://cs231n.github.io/transfer-learning/) - там есть огромный ImageNet, на котором предобучают модель, чтобы потом заморозить нижние слои и заменить выходные. В итоге модель использует универсальные представления данных, выученные на большом корпусе, но для предсказания совсем других меток - и качество очень здорово растет.

Нам такие извращения пока не нужны (хотя потом пригодятся, ключевые слова: ULMFiT, ELMo и компания). Просто возьмем обученную на большем корпусе модель и поучим ее на меньшем корпусе. Ей всего-то нужно новый матрический шаблон последней строки выучить.

**Задание** Обученную в прошлом пункте модель дообучите на порошки.

### Conditional language model

Ещё лучше - просто учиться на обоих корпусах сразу. Объедините пирожки и порошки, для каждого храните индекс 0/1 - был ли это пирожок или порошок. Добавьте вход - этот индекс и конкатенируйте его либо к каждому эмбеддингу слов, либо к каждому выходу из LSTM.

**Задание** Научите единую модель, у которой можно просить сгенерировать пирожок или порошок.

### Variational & word dropout

**Задание** На прошлом занятии приводились примеры более приспособленных к RNN'ам dropout'ов. Добавьте их.

**Задание** Кроме этого, попробуйте увеличивать размер модели или количество слоев в ней, чтобы улучшить качество.

## Multi-task learning

Ещё один важный способ улучшения модели - multi-task learning. Это когда одна модель учится делать предсказания сразу для нескольких задач.

В нашем случае это может быть предсказанием отдельно леммы слова и отдельно - его грамматического значения:
![](https://hsto.org/web/e97/8a8/6e8/e978a86e8a874d8d946bb15e6a49a713.png =x350)

В итоге модель выучивает как языковую модель по леммам, так и модель POS tagging'а. Одновременно!

Возьмем корпус из universal dependencies - он уже размечен, как нужно.

Почитаем его:

In [0]:
from corpus_iterator import Token, CorpusIterator

fields = [('word', Field()), ('lemma', Field()), ('gram_val', Field())]
examples = []

with CorpusIterator('UD_Russian-SynTagRus/ru_syntagrus-ud-train.conllu') as corpus_iter:
    for sent in corpus_iter:
        words = ['<s>'] + [tok.token.lower() for tok in sent] + ['</s>']
        lemmas = ['<s>'] + [tok.lemma.lower() for tok in sent] + ['</s>']
        gr_vals = ['<s>'] + [tok.grammar_value for tok in sent] + ['</s>']
        examples.append(Example.fromlist([words, lemmas, gr_vals], fields))

In [0]:
print('Words:', examples[1].word)
print('Lemmas:', examples[1].lemma)
print('Grammar vals:', examples[1].gram_val)

Таким образом, размер словаря может быть существенно сокращен - лемм меньше, чем слов, а предсказание грамматики вынуждает модель быть более осведомленной о согласовании слов.

In [0]:
dataset = Dataset(examples, fields)

dataset.fields['word'].build_vocab(dataset, min_freq=3)
print('Word vocab size =', len(dataset.fields['word'].vocab))
dataset.fields['lemma'].build_vocab(dataset, min_freq=3)
print('Lemma vocab size =', len(dataset.fields['lemma'].vocab))
dataset.fields['gram_val'].build_vocab(dataset)
print('Grammar val vocab size =', len(dataset.fields['gram_val'].vocab))

train_dataset, test_dataset = dataset.split(split_ratio=0.75)

train_iter, test_iter = BucketIterator.splits(datasets=(train_dataset, test_dataset), batch_sizes=(32, 128), 
                                              shuffle=True, device=DEVICE, sort=False)

Построим маппинг из пары (лемма, грамматическое значение) в слово - если бы у нас под рукой был морфологический словарь, маппинг можно было бы пополнить, добавить слова для лемм из корпуса, которые не встретились в обучении.

In [0]:
dictionary = {
    (lemma, gr_val): word
    for example in train_iter.dataset.examples 
    for word, lemma, gr_val in zip(example.word, example.lemma, example.gram_val)
}

**Задание**  Обновите генератор - например, можно сэмплировать лемму и находить самое вероятное грамматическое значение, которое встречается  в паре с этой леммой в `dictionary`.

In [0]:
def generate(model, temp=0.7):
    ...

**Задание** Обновите модель и функцию обучения.

Модель должна принимать пары `lemma, gr_val`, конкатенировать их эмбеддинги и предсказывать следующие `lemma, gr_val` по выходу из LSTM.

Функция `do_epoch` должна суммировать потери по предсказанию леммы (делая маскинг для `<unk>` и `<pad>`) + потери по предсказанию грамматического значения (с маскингом по `<pad>`).

## Контролируемая генерация

Хочется сделать генерацию более контролируемой - в идеале, задавать тему.

Простой способ - сделать тематическое моделирование и найти в текстах какие-то темы - а потом передавать вектор тем вместе с эмбеддингом слова, чтобы модель училась генерировать тематически-согласованный текст.

In [0]:
from gensim import corpora, models

docs = [[word for word in poem if word != '\\n'] for poem in perashki]

dictionary = corpora.Dictionary(docs)
dictionary.filter_n_most_frequent(100)

bow_corpus = [dictionary.doc2bow(doc) for doc in docs]

lda_model = models.LdaModel(bow_corpus, num_topics=5, id2word=dictionary, passes=5)

Посмотреть, что выучилось, можно так:

In [0]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)

Предсказывает распределение модель как-то так:

In [0]:
for word in perashki[10]:
    if word == '\\n':
        print()
    else:
        print(word, end=' ')

In [0]:
lda_model.get_document_topics(bow_corpus[10])

**Задание** Посчитайте для всех текстов вектора тем, передавайте их вместе со словами (конкатенируя к эмбеддингам). Посмотрите, вдруг чего получится.

# Дополнительные материалы

## Статьи

Regularizing and Optimizing LSTM Language Models, 2017 [[arxiv]](https://arxiv.org/abs/1708.02182), [[github]](https://github.com/salesforce/awd-lstm-lm) - одна из самых полезных статей про языковые модели + репозиторий, в котором реализовано много полезного, стоит заглянуть

Exploring the Limits of Language Modeling, 2016 [[arxiv]](https://arxiv.org/abs/1602.02410)

Using the Output Embedding to Improve Language Models, 2017 [[pdf]](http://www.aclweb.org/anthology/E17-2025)


## Transfer learning
[Transfer learning, cs231n](http://cs231n.github.io/transfer-learning/)  
[Transfer learning, Ruder](http://ruder.io/transfer-learning/) - очень подробная статья от чувака из NLP

## Multi-task learning
[An Overview of Multi-Task Learning in Deep Neural Networks, Ruder](http://ruder.io/multi-task/)  
[Multi-Task Learning Objectives for Natural Language Processing, Ruder](http://ruder.io/multi-task-learning-nlp/)

# Сдача

[Форма для сдачи](https://goo.gl/forms/ASLLjYncKUcIHmuO2)  
[Feedback](https://goo.gl/forms/9aizSzOUrx7EvGlG3)